In [1]:
import collections
%matplotlib inline
import matplotlib
import numpy as np
import openml
import pandas as pd
import time
print(openml.config.server)

https://www.openml.org/api/v1/xml


In [2]:
automlbench_dids = [41021, 42225, 42571, 4549, 42572, 42570, 42705,
                   42728, 550, 546, 541, 507, 505, 287, 216, 41540,
                   42688, 422, 416, 42724, 42727, 42729, 42726, 42730,
                   201, 41980, 42731, 531, 42563, 574, 3050, 3277, 43071]

In [3]:
all_datasets = openml.datasets.list_datasets()
to_consider = openml.datasets.list_datasets()
TOTAL = len(all_datasets)
# print(TOTAL)
# # all_tasks = openml.tasks.list_tasks()
to_remove = set()
# # print(len(all_tasks))
# print("From:", len(to_consider)) # 3004

In [4]:
# remove datasets from the datasets list which are in the automl bench or close relatives
print("From:", len(to_consider))

# remove exact duplicates
for did in all_datasets:
    if did in automlbench_dids:
        to_remove.add(did)

for did in to_remove:
    if did in to_consider:
        del to_consider[did]
assert len(to_consider) == len(all_datasets) - len(automlbench_dids)

print("To:", len(to_consider)) # 2965

From: 5013
To: 4980


In [5]:
# Remove datasets with too few features, instances or classes
print("From:", len(to_consider))

for did in all_datasets:
    try:
        if all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] < 1_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] >= 500_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] < 20:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] < 2:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] > 2500:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfClasses'] > 1:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] > 10_000_000:
            to_remove.add(did)
    except KeyError:
        #print(did, all_datasets[did])
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 651

From: 4980
To: 1546


In [6]:
all_datasets_as_frame = pd.DataFrame.from_dict(all_datasets, orient='index')

In [7]:
# Remove all datasets with "BNG" in name
print("From:", len(to_consider))

for did in to_consider:
    if to_consider[did]['name'].startswith("BNG"):
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 641

From: 1546
To: 1539


In [8]:
# check for similar names and manually check overlaps
# def return_triplets(name):
#     triplets = set()
#     name = name.lower()
#     for i in range(len(name) - 2):
#         triplets.add(name[i: i+3])
#     return triplets

# manually_look_at = []
# for did in to_consider:
#     for did2 in automlbench_dids:
#         automl_triplets = return_triplets(all_datasets[did2]['name'])
#         other_triplets = return_triplets(all_datasets[did]['name'])
#         n_hits = sum([trip in other_triplets for trip in automl_triplets])
#         ratio = n_hits / len(automl_triplets)
#         if n_hits > 2:
#             manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

# df = pd.DataFrame(manually_look_at)
# with open("Manualautomlregr.csv", "w") as fh:
#    fh.write(df.to_csv())     


In [9]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1XH1mQFq7-1fs28F-CQJo6i6Wg7IlpDS0aDTAJBoaD7k/edit#gid=0
print("From:", len(to_consider))
for did in [209, 543, 4545, 42092, 42635, 42723, 43465, 43984, 43986, 
            43989, 43992, 43994, 43996, 43997, 43998, 43999, 44002, 
            44009, 44011, 44013, 44014, 44015, 44016, 44019, 44039, 
            44042, 44044, 44047, 44049, 44051, 44054, 44057, 44059, 
            44061, 44062, 44063, 44064, 44066, 44134, 44136, 44138, 
            44139, 44140, 4414, 44142, 44144, 44152, 44201, 44202, 
            44203, 44269, 44956, 44979, 44985, 44987, 44988, 44990, 
            45031, 45032, 45033, 45041, 45042, 42713, 44000, 44017, 44048, 44063, 44142]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 603

From: 1539
To: 1474


In [10]:
# # check for similar metafeatures and manually check overlaps
# metafeature_names = [
#     'MaxNominalAttDistinctValues', 'NumberOfClasses',
#     'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
#     'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
# ]

# simple_metafeatures_to_consider = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in to_consider}
# ).transpose()[metafeature_names]
# simple_metafeatures_automl = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in automlbench_dids}
# ).transpose()[metafeature_names]

# # Checking the hamming distance of the datasets
# manually_look_at = []
# for did1, mf1 in simple_metafeatures_to_consider.iterrows():
#     for did2, mf2 in simple_metafeatures_automl.iterrows():
#         n_hits = np.sum(mf1 == mf2)
#         ratio = n_hits / len(mf1)
#         n_hits = np.sum(
#             mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
#         )
#         if ratio > 0.5 or n_hits == 2:
#             entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
#             mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
#             entry.extend(mfs)
#             manually_look_at.append(entry)

# header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
# mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
# df = pd.DataFrame(manually_look_at, columns = header + mfs)
# # with open("ManualMetafeaturesregr.csv", "w") as fh:
# #    fh.write(df.to_csv())            

In [11]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1SWEBsFTykdUMq-wBcx9NHchhdolB632EJYaAmQwwQtM/edit#gid=0
print("From:", len(to_consider))
for did in [315, 580, 610, 652, 41938, 41968, 41969, 43070, 43672, 43888, 43983, 43991,
            44008, 44133, 44141, 44971, 44972, 44989, 45030]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 599
# check whether we can work usefully with creditcard (1597)

From: 1474
To: 1455


In [12]:
pd.set_option('display.max_rows', 25)
columns = ['name', 'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances', ]

to_consider_stats = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()[columns]

to_consider_stats

,name,NumberOfClasses,NumberOfFeatures,NumberOfInstances
8,liver-disorders,0.0,6.0,345.0
189,kin8nm,0.0,9.0,8192.0
191,wisconsin,0.0,33.0,194.0
194,cleveland,0.0,14.0,303.0
195,auto_price,0.0,16.0,159.0
...,...,...,...,...
45075,KDD,0.0,46.0,5032.0
45076,churn,0.0,21.0,5000.0
45078,nomao,0.0,119.0,34465.0
45079,road-safety,0.0,33.0,111762.0


In [13]:
# remove sparse datasets
# print("From:", len(to_consider))
# start = time.time()
# for i, did in enumerate(to_consider):
#     #print(did)
#     if (i+1) % 25 == 0:
#         print("%4.2fsec: %d/%d" % (time.time() - start, i+1, len(to_consider)))
#     d = openml.datasets.get_dataset(int(did), download_data=False)
#     #print(
#     #    d.format.lower(), 
#     #    'sparse' in d.format.lower(), 
#     #    d.description and 'CLASSINDEX: none specific' in d.description
#     #)
#     if 'sparse' in d.format.lower():
#         to_remove.add(did)
#     if d.description and 'CLASSINDEX: none specific' in d.description:
#         to_remove.add(did)
#     for f in d.features.values():
#         if f.data_type == 'string':
#             to_remove.add(did)

# for did in to_remove:
#     if did in to_consider:
#         del to_consider[did]
# print("To:", len(to_consider)) # 599

In [14]:
to_consider

OrderedDict([(8,
              {'did': 8,
               'name': 'liver-disorders',
               'version': 1,
               'uploader': '1',
               'status': 'active',
               'format': 'ARFF',
               'NumberOfClasses': 0.0,
               'NumberOfFeatures': 6.0,
               'NumberOfInstances': 345.0,
               'NumberOfInstancesWithMissingValues': 0.0,
               'NumberOfMissingValues': 0.0,
               'NumberOfNumericFeatures': 6.0,
               'NumberOfSymbolicFeatures': 0.0}),
             (189,
              {'did': 189,
               'name': 'kin8nm',
               'version': 1,
               'uploader': '1',
               'status': 'active',
               'format': 'ARFF',
               'NumberOfClasses': 0.0,
               'NumberOfFeatures': 9.0,
               'NumberOfInstances': 8192.0,
               'NumberOfInstancesWithMissingValues': 0.0,
               'NumberOfMissingValues': 0.0,
               'NumberOfNumeric

In [15]:
# remove identical/similar datasets (to reduce the load computing the meta-data)
# check for similar names and manually check overlaps
def return_triplets(name):
    triplets = set()
    name = name.lower()
    for i in range(len(name) - 2):
        triplets.add(name[i: i+3])
    return triplets

#Remove QSAR datasets

manually_look_at = []
for i, did in enumerate(to_consider):
    for j, did2 in enumerate(to_consider):
        
        if all_datasets[did]["name"].startswith("QSAR") or all_datasets[did2]["name"].startswith("QSAR"):
            continue
        if j <= i:
            continue
        if len(all_datasets[did2]['name']) < 3:
            continue
        other2_triplets = return_triplets(all_datasets[did2]['name'])
        other_triplets = return_triplets(all_datasets[did]['name'])
        n_hits = sum([trip in other_triplets for trip in other2_triplets])
        ratio = n_hits / len(other2_triplets)
        if n_hits > 2:
            manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

df = pd.DataFrame(manually_look_at)
with open("manual_check.csv", "w") as fh:
   fh.write(df.to_csv())   

PermissionError: [Errno 13] Permission denied: 'manual_check.csv'

In [18]:
df

,0,1,2,3,4,5
0,3,0.093750,liver-disorders,The-Estonia-Disaster-Passenger-List,8,43389
1,5,0.135135,liver-disorders,Indian-Liver-Patient-Patient-Records-KFolds-5f...,8,43403
2,5,0.192308,liver-disorders,Indian-Liver-Patient-Records,8,43665
3,5,0.192308,liver-disorders,Indian-Liver-Patient-Records,8,43675
4,3,0.088235,liver-disorders,Heart-Disease-Dataset-(Comprehensive),8,43682
5,4,1.000000,kin8nm,kin8nm,189,44980
6,3,0.333333,cleveland,ozone_level,194,301
7,5,0.714286,auto_price,autoPrice,195,207
8,4,0.400000,auto_price,house_prices,195,42165
9,3,0.176471,auto_price,Lisbon-House-Prices,195,43660


In [35]:
with open("dataset_to_dataset_check_regr.csv", "w") as fh:
   fh.write(df.to_csv())          

In [50]:
dataset = openml.datasets.get_dataset(44965)
X, y, categorical_indicator, attribute_names = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)

In [49]:
X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118
0,7.161286,7.835325,2.911583,0.984049,-1.499546,-2.094097,0.576000,-1.205671,1.849122,-0.425598,...,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-15.75,-47.95
1,0.225763,-0.094169,-0.603646,0.497745,0.874036,0.290280,-0.077659,-0.887385,0.432062,-0.093963,...,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,14.91,-23.51
2,-0.692525,-0.517801,-0.788035,1.214351,-0.907214,0.880213,0.406899,-0.694895,-0.901869,-1.701574,...,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,12.65,-8.00
3,-0.735562,-0.684055,2.058215,0.716328,-0.011393,0.805396,1.497982,0.114752,0.692847,0.052377,...,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,9.03,38.74
4,0.570272,0.273157,-0.279214,0.083456,1.049331,-0.869295,-0.265858,-0.401676,-0.872639,1.147483,...,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,34.03,-6.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,0.399577,0.310805,-0.039326,-0.111546,0.304586,-0.943453,0.114960,-0.335898,0.826753,-0.393786,...,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-6.17,35.74
1055,1.640386,1.306224,0.192745,-1.816855,-1.311906,-2.128963,-1.875967,0.094232,-1.429742,0.873777,...,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,11.55,104.91
1056,-0.772360,-0.670596,-0.840420,-0.832105,0.277346,1.152162,0.241470,0.229092,0.019036,-0.068804,...,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,41.33,19.80
1057,-0.996965,-1.099395,3.515274,-0.508185,-1.102654,0.192081,0.069821,0.264674,-0.411533,0.501164,...,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,54.68,25.31


In [51]:
X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116
0,7.161286,7.835325,2.911583,0.984049,-1.499546,-2.094097,0.576000,-1.205671,1.849122,-0.425598,...,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194
1,0.225763,-0.094169,-0.603646,0.497745,0.874036,0.290280,-0.077659,-0.887385,0.432062,-0.093963,...,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616
2,-0.692525,-0.517801,-0.788035,1.214351,-0.907214,0.880213,0.406899,-0.694895,-0.901869,-1.701574,...,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755
3,-0.735562,-0.684055,2.058215,0.716328,-0.011393,0.805396,1.497982,0.114752,0.692847,0.052377,...,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169
4,0.570272,0.273157,-0.279214,0.083456,1.049331,-0.869295,-0.265858,-0.401676,-0.872639,1.147483,...,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054,0.399577,0.310805,-0.039326,-0.111546,0.304586,-0.943453,0.114960,-0.335898,0.826753,-0.393786,...,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247,-0.415247
1055,1.640386,1.306224,0.192745,-1.816855,-1.311906,-2.128963,-1.875967,0.094232,-1.429742,0.873777,...,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538,-0.817538
1056,-0.772360,-0.670596,-0.840420,-0.832105,0.277346,1.152162,0.241470,0.229092,0.019036,-0.068804,...,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309,-0.515309
1057,-0.996965,-1.099395,3.515274,-0.508185,-1.102654,0.192081,0.069821,0.264674,-0.411533,0.501164,...,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855,0.074855


In [56]:
all_datasets_as_frame

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
43384,43384,Diabetes-Data-Set,1,30126,active,arff,NaN,NaN,NaN,0.0,9.0,768.0,0.0,0.0,9.0,0.0
43483,43483,Pima-Indians-Diabetes-Dataset,1,30126,active,arff,NaN,NaN,NaN,0.0,9.0,768.0,0.0,0.0,9.0,0.0
43582,43582,Pima-Indians-Diabetes,1,30123,active,arff,NaN,NaN,NaN,0.0,9.0,768.0,0.0,0.0,9.0,0.0
43874,43874,Diabetes-130-Hospitals_(Fairlearn),1,30211,active,arff,NaN,NaN,NaN,0.0,25.0,101766.0,0.0,0.0,7.0,5.0


In [19]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/12Tmtgrz_92fuOKoIIIb6ROGkd65nv7R3g7EZAYwtTos/edit#gid=0
print("From:", len(to_consider))
for did in [45013, 45014, 44980, 195, 42372, 573, 43981, 43982, 43990,
            44007, 44132, 45029, 44978, 43882, 42900, 562, 43987, 43995, 44012, 44137,
            555, 556, 44055, 43127, 504, 568, 570, 575, 576, 577, 578, 670, 671, 672,
            44041, 44056, 43056, 1436, 1435, 44965, 44003, 44020, 23515,42436, 42437, 
            42438, 42439, 42444, 42445, 41514, 41515, 41516, 41517, 41518, 41519, 43384, 
            43483, 43582, 43943, 43959, 44187, 44191, 44192, 44194, 44223, 41516, 41517, 43123, 
            41523, 41524, 41525, 43127, 44236, 41928, 42131] :
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 503

From: 1455
To: 1455


In [21]:
# Subsample from 'topics', where there are dozens of super-similar
# datasots from one source
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider}
# Set random number generator to always sample the same set
# of datasets
rng = np.random.RandomState(1)
# Friedman datasets
friedman_datasets = [
    did for did, name in dataset_names.items() if name.startswith('fri_c')
]
# ova & ap datasets
ova_datasets = [
    did for did, name in dataset_names.items() 
    if name.startswith('OVA_') or name.startswith('AP_')
]
# volcanoes datasets
volcanoes_datasets = [
    did for did, name in dataset_names.items() 
    if name.startswith('volcanoes-')
]
# AutoUniv
auto_univ_datasets = [
    did for did, name in dataset_names.items() 
    if name.startswith('autoUniv-')
]
for dataset_ids_to_sample_from, num_keep in (
    (friedman_datasets, 10),
    (ova_datasets, 7),
    (volcanoes_datasets, 10),
    (auto_univ_datasets, 4),
):
    choices_to_drop = np.random.choice(
        dataset_ids_to_sample_from,
        replace=False,
        size=len(dataset_ids_to_sample_from) - num_keep)
    for choice in choices_to_drop:
        if choice in to_consider:
            del to_consider[choice]
print("To:", len(to_consider)) # 454

To: 1337


In [22]:
# Delete a few other datasets

# Click prediction datasets consist almost exclusively of IDs
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider
                 and value['name'].startswith('Click_prediction')}
for did in dataset_names:
    if did in to_consider:
        del to_consider[did]

dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider
                 and value['name'].startswith('FOREX')}
for did in dataset_names:
    if did in to_consider:
        del to_consider[did]


# Forex datasets are time series dataset with a time stamp
# regularly shuffled tasks won't work here
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider
                 and value['name'].startswith('FOREX')}
for did in dataset_names:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 253

To: 1134


In [23]:
print("From:", len(to_consider))
to_remove.update([
    312,  # multilabel
    313,  # hierarchical classification
    316,  # multilabel classification
    378,  # unclear target
    381,  # unclear target
    382,  # unclear target
    1178,  # multilabel
    1179,  # multilabel
    1472,  # regression
    1477,  # not a regular classification dataset (stream)
    40588,  # multilabel
    40589,  # multilabel
    40590,  # multilabel
    40591,  # multilabel
    40592,  # multilabel
    40593,  # multilabel
    40594,  # multilabel
    40595,  # multilabel
    40596,  # multilabel
    40597,  # multilabel
    40686,  # multilabel
    40687,  # multilabel
    40702,  # multilabel
    40910,  # stream dataset
    41103,  # description says 'CIFAR-10 dataset but with some modifications'
    41526,  # is named test_dataset
])
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 232

From: 1134
To: 1112


In [24]:
# check for similar metafeatures and manually check overlaps
metafeature_names = [
    'MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize', 'NumberOfClasses',
    'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
    'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
]

simple_metafeatures_to_consider = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()[metafeature_names]

# Checking the hamming distance of the datasets
manually_look_at = []
for did1, mf1 in simple_metafeatures_to_consider.iterrows():
    for did2, mf2 in simple_metafeatures_to_consider.iterrows():
        if did2 <= did1:
            continue
        n_hits = np.sum(mf1 == mf2)
        ratio = n_hits / len(mf1)
        n_hits = np.sum(
            mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
        )
        if ratio > 0.5 or n_hits == 2:
            entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
            mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
            entry.extend(mfs)
            manually_look_at.append(entry)

header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
df = pd.DataFrame(manually_look_at, columns = header + mfs)
#with open("/tmp/ManualMetafeatures.csv", "w") as fh:
#    fh.write(df.to_csv()) 

KeyboardInterrupt: 

In [25]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1qVqH_ZVWJ3Eu0B6vXF0q2eQDMwiU5LXToN1L9ZfgLIs/edit#gid=0
print("From:", len(to_consider))
for did in [983, 38, 40707, 40708, 40713, 40690, 454, 41156,
            40678, 41964]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 222

From: 1112
To: 1102


In [49]:
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider}
for did in [
    914,  # Balloon dataset, has only a single features
    993,  # not a classification dataset
    1002,  # not a classification dataset
    1018,  # not a classification dataset
    40497,  # regular thyroid dataset
    40517,  # artificial drift detection dataset
    40666,  # derived from the musk dataset (1116) we use
    41158,  # derived from MNIST
    41960,  # appears to not be a classification dataset
    42344,  # appears to not be a classification dataset
]:
    to_remove.add(did)

for did in [ # recreated samples from automl benchmark datasets
    44593, 
    44498, 
    44557, 
    44618, 
    44780, 
    44698, 
    44729, 
    44535
]:
    to_remove.add(did)
    
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 209

To: 1086


In [50]:
print(len(to_consider), len(to_remove), TOTAL)

1086 3631 5012


In [51]:
final = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()

In [52]:
final_filtered = final.groupby(['NumberOfClasses', 'NumberOfInstances']).sample(n = 1)

In [ ]:
# dataset_ids = list(to_consider)
# datasets_to_tasks = collections.defaultdict(list)
# for task_id, task in all_tasks.items():
#     if task['ttid'] != 1:
#         continue
#     if task['estimation_procedure'] != '33% Holdout set':
#         continue
#     if task['did'] not in to_consider:
#         continue
#     datasets_to_tasks[task['did']].append(task_id)

In [ ]:
# Datasets which not yet have a 33% holdout task associated!
# datasets_without_task = sorted(list(set(to_consider) - set(datasets_to_tasks)))
# for dataset_wo_task in datasets_without_task:  
#     try:
#         d = openml.datasets.get_dataset(int(dataset_wo_task), download_data=False)            
#     except ValueError as e:
#         print(e)
#         continue
#     print(dataset_wo_task)

In [ ]:
# # Convert the 10-fold CV OpenML CC18 tasks into holdout tasks
# cc18_did_to_task_id = dict()
# for did in automlbench_dids:
#     if did not in cc18_did_to_task_id:
#         cc18_did_to_task_id[did] = list()
#     for task_id, task in all_tasks.items():
#         if task['ttid'] != 1:
#             continue
#         if task['estimation_procedure'] != '33% Holdout set':
#             continue
#         if task['did'] != did:
#             continue
#         #if 'evaluation_measures' in task:
#         #    continue
#         cc18_did_to_task_id[task['did']].append(task_id)
# print(len(cc18_did_to_task_id))
# #print(cc18_did_to_task_id)
# print([l[0] for l in list(cc18_did_to_task_id.values())])

39
[233, 242, 261, 283, 75099, 75105, 75127, 75193, 126025, 126026, 126029, 146587, 146589, 75227, 75097, 167083, 167104, 75101, 146680, 168792, 168793, 168794, 168795, 168796, 168797, 168798, 189860, 189861, 189862, 189865, 189866, 189871, 189872, 189873, 189874, 168786, 168787, 168789, 168790]
